## Подготовка данных для занятия

In [7]:
import re

import pandas as pd
import pymorphy2
import numpy as np

text_df = pd.read_csv("./data/content_description.csv", sep='\t')
print('text_df\n', text_df.head().to_string())

# разбиваем тест на слова
corpus = []
# регулярка для поиска слов
regular_expr = r'\w+'
reg_expr_compiled = re.compile(regular_expr)
# формируем датасет из отдельных слов
for raw_text in text_df.description.values:
    # приводим к нижнему регистру
    raw_text_lower = raw_text.lower()
    # разбиваем текст на слова
    text_by_words = reg_expr_compiled.findall(raw_text_lower) 
    corpus.append(text_by_words)

# нормализация текста
normalized_corpus = []
morph = pymorphy2.MorphAnalyzer()
# нормализуем каждое слово в тексте
for token_list in corpus:
    normalized_token_list = []
    for word in token_list:
        parsed_token = morph.parse(word)
        normal_form = parsed_token[0].normal_form
        normalized_token_list.append(normal_form)
    normalized_corpus.append(normalized_token_list)

# превращаем в DataFrame
doc_count = len(normalized_corpus)
doc_ids = []
tokens = []
# формируем два списка-колонки датафрейма
for doc_id in range(doc_count):
    for token in normalized_corpus[doc_id]:
        doc_ids.append(doc_id)
        tokens.append(token)

tokens_df = pd.DataFrame({
    'doc_id': doc_ids,
    'word': tokens
})
# дамми-столбец
tokens_df = tokens_df.assign(dummy = 1)
# аггрегируем статистики
word_count_df = tokens_df.groupby(['doc_id','word'])['dummy'].count().reset_index()

print('\nword_count_df\n',
    word_count_df[word_count_df.doc_id==0].sort_values(by='dummy', ascending=False).head(10).to_string()
)

text_df
                                        content                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 

# 10.3  Домашняя работа

10.3.1 **Задание простого уровня** Для каждого слова подсчитайте процент документов, в которых содержится это слово. Сформируйте dataframe doc_frequency. Процен документов вычисляется по формуле
$$
d = \frac{m}{n} \times 100
$$
где $m$ - количество документов, в которых встретилось это слово, а $n$ - общее количество документов. Значение в процентах округлите до целых.

Посмотрите какие слова оказались в топе - это предлоги и частицы. Такие слова присутствуют во всех документах, а значит, их можно удалить из текста, чтобы оставшиеся слова были более "осмысленными". Это упростит анализ текста.

In [8]:
# -- ВАШ КОД ЗДЕСЬ --

# doc_frequency = pd.DataFrame()
doc_frequency = tokens_df.groupby('word')['doc_id'].unique().to_frame()
# doc_frequency_count.head(10)
for idx in doc_frequency.index:
    doc_frequency.loc[idx,'count'] = len(doc_frequency.loc[idx,'doc_id'])
doc_id_number = len(tokens_df['doc_id'].unique())
doc_frequency['percent'] = np.around(doc_frequency['count'] / doc_id_number * 100).astype(int)
# print(doc_frequency.head(10))
doc_frequency.sort_values(by='percent', ascending=False).head(10)

,doc_id,count,percent
word,,,
и,"[0, 1, 2, 3, 4, 5, 6, 7]",8.0,100
в,"[0, 1, 2, 3, 4, 5, 6, 7]",8.0,100
на,"[0, 1, 3, 4, 5, 6, 7]",7.0,88
с,"[0, 1, 2, 3, 4, 5, 7]",7.0,88
он,"[0, 1, 2, 3, 4, 7]",6.0,75
не,"[1, 2, 3, 4, 6, 7]",6.0,75
свой,"[0, 1, 2, 3, 5, 6]",6.0,75
онлайн,"[0, 1, 3, 4, 6, 7]",6.0,75
что,"[0, 1, 3, 4, 5, 6]",6.0,75


10.3.2 **Задание среднего уровня**

Воспользуйтесь регулярными выражениями, чтобы извлечь из текста все пары **имя+фамилия**.

* неформальное описание регулярки: пара слов идущая друг за другом, каждое из которых начинается с заглавной буквы
* анализировать нужно только `doc_id=3`
* текст берём из исходного датафрейма `text_df`
* заглавная буква в русскоязычном тексте соответствует символьному классу `r'[A-Я]*'`

In [9]:
# -- ВАШ КОД ЗДЕСЬ --
raw_text = text_df.description.values[3]
# print(raw_text)
# регулярка - её нужно поправить
# reg_expr = r'\. \w+'
reg_expr = r'(?<!\.\s)[А-Я]\w+\s+[А-Я]\w+'
# компилируем регулярное выражение
reg_expr_compiled = re.compile(reg_expr)
# применяем выражение к тексту
for g in reg_expr_compiled.findall(raw_text):
    print(g)

Стивена Фрая
Джоном Дженксом
Теда Уоллеса


10.3.3 **Задание сложного уровня**. Словарь `genre_dict` содержит слова, которые являются характерными для того или иного жанра. Пользуясь словарём, а так же таблицей `word_count_df`, сформируйте таблицу двумя колонками `doc_id | genre` с жанрами фильмов.

Для этого нужно
* превратить словарь `genre_dict` в DataFrame формата `word | genre`
* соединить полученный датафрейм с помощью функции `merge` c ранее полученным датафреймом `word_count_df`, котороый содержит распределение слов по документам. Воспользуйтесь методом соединения `inner`
* для каждого документа выбрать жанр документа - это совокупность жанров слов отдельных слов. У одного контента может быть несколько жанров

In [10]:
import itertools
import pandas as pd

genre_dict = {
    'комедия': ['сатирический', 'авантюрный', 'забавный'],
    'мелодрама': ['выбор', 'позор'],
    'сказка': ['приключения', 'милый', 'семейный'],
    'детектив': ['тайна', 'разгадать', 'загадочный'], 
    'триллер': ['ужас', 'зловещий', 'нерв']
}

nested_genres =[[(i, j) for j in genre_dict[i]] for i in genre_dict]
# переводим из словаря к более удобному виду
flatten_genres = list(itertools.chain(*nested_genres))
# создаём DataFrame
genres_df = pd.DataFrame(flatten_genres, columns = ['genre', 'character_word'])
# print(genres_df.head())

# -- ВАШ КОД ЗДЕСЬ --
doc_to_genres = pd.DataFrame()
doc_to_genres = word_count_df.merge(genres_df, how='inner', left_on='word', right_on='character_word' )[['doc_id', 'genre']]
# print(doc_to_genres)
print(doc_to_genres.groupby('doc_id')['genre'].unique().to_frame())
print(doc_to_genres.drop_duplicates().sort_values(by='doc_id').set_index('doc_id'))

                      genre
doc_id                     
0       [детектив, триллер]
1                  [сказка]
2               [мелодрама]
3                 [комедия]
4                 [комедия]
6                 [триллер]
            genre
doc_id           
0        детектив
0         триллер
1          сказка
2       мелодрама
3         комедия
4         комедия
6         триллер
